# Mapping

Mappings are pre-defined configuration files that encode the logic on how to transform a specific data source into Resources that follow a template of a targeted _Type_. 

This notebook demonstrates the `DictionaryMapping` wich is based on a JSON structure that represent the target structure, and Python code that will apply desired transformations on the data source.

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [ ]:
from kgforge.core import Resource

In [ ]:
from kgforge.specializations.mappings import DictionaryMapping

## Data

In [ ]:
scientists = [
    {
        "id": 123,
        "name": "Marie Curie",
        "gender": "female",
        "middle_name": "Salomea",
    },
    {
        "id": 456,
        "name": "Albert Einstein",
        "gender": "male",
        "middle_name": "(missing)",
    },
]

## Mapping data to the Knowledge Graph Schema

### basics

Note: DemoModel and RdfModel schemas have not been synchronized yet. The following cell is to be run with DemoModel.

In [ ]:
forge.template("Association")

In [ ]:
mapping_simple = DictionaryMapping("""
    type: Association
    agent:
    {
        type: Person
        name: x.name
    }
""")

In [ ]:
resources_simple = forge.map(scientists, mapping_simple)

In [ ]:
print(resources_simple[0])

### missing values

In [ ]:
mapping_na = DictionaryMapping("""
    type: Association
    agent:
    {
        type: Person
        name: x.name
        additionalName: x.middle_name
    }
""")

In [ ]:
print(forge.map(scientists[1], mapping_na))

In [ ]:
print(forge.map(scientists[1], mapping_na, na="(missing)"))

### multiple mappings

In [ ]:
mapping_person = DictionaryMapping("""
    id: forge.format("identifier", "persons", x.id)
    type: Person
    name: x.name
""")

In [ ]:
mapping_association = DictionaryMapping("""
    type: Association
    agent: forge.format("identifier", "persons", x.id)
""")

In [ ]:
resources_graph = forge.map(scientists, [mapping_person, mapping_association])

In [ ]:
print(resources_graph[0])

In [ ]:
print(resources_graph[1])

### managed mappings

Note: RdfModel doesn't implement managed mappings operations yet. Please use DemoModel for this section.

In [ ]:
forge.sources()

In [ ]:
forge.mappings("scientists-database")

In [ ]:
mapping = forge.mapping("Association", "scientists-database")

In [ ]:
resources = forge.map(scientists, mapping, na="(missing)")

In [ ]:
type(resources)

In [ ]:
type(resources[0])

In [ ]:
print(mapping)

In [ ]:
print(resources[0])

## Managing mappings

Note: RdfModel doesn't implement managed mappings operations yet. Please use DemoModel for this section.

In [ ]:
filepath = "mappings/scientists-database/DictionaryMapping/Association.hjson"

### saving

In [ ]:
mapping.save(filepath)

In [ ]:
filepath

### tracking & sharing changes

In [ ]:
# ! cd mappings

In [ ]:
# ! git add Association.hjson

In [ ]:
# ! git commit -m "Add Association mapping"

In [ ]:
# ! git push

### loading

In [ ]:
loaded = DictionaryMapping.load(filepath)

In [ ]:
# loaded == mapping

In [ ]:
# ! rm -R ./mappings/